In [1]:
import os
import tensorflow as tf
if tf.__version__!='2.11.0':
  !pip install tensorflow==2.11.0 --quiet
  # os.kill(os.getpid(), 9)

In [2]:
!pip install keras-rl2 --quiet
!pip install gym[atari] --quiet
!pip install atari-py --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!python -m atari_py.import roms /content/drive/MyDrive/dqn/roms/

/usr/bin/python3: No module named atari_py.import


In [10]:
import gym
from gym.envs.registration import register

In [11]:
register(
    id='Breakout-v4',
    entry_point='gym.envs.atari:AtariEnv',
    kwargs={'game': 'breakout', 'obs_type': 'image', 'frameskip': 1},
    max_episode_steps=10000,
    nondeterministic=False,
)

In [9]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.callbacks import Callback
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [6]:
ENV_NAME = 'Breakout-v4'

env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

class CustomCallback(Callback):
    def on_step_end(self, step, logs={}):
        if step % 100 == 0:
            print('Average reward at step {}: {}'.format(step, logs.get('episode_reward')))

dqn.fit(env, nb_steps=5000, visualize=False, verbose=2, callbacks=[CustomCallback()])

dqn.save_weights('/content/drive/MyDrive/dqn/weights.h5', overwrite=True)

ImportError: cannot import name 'AtariProcessor' from 'rl.processors' (/usr/local/lib/python3.10/dist-packages/rl/processors.py)

In [ ]:
dqn.load_weights('/content/drive/MyDrive/dqn/weights.h5')

dqn.test(env, nb_episodes=5, visualize=True)